# Integration

Zhentao Shi

<!-- code is tested on SCRP -->

* Mathematically, integration and differentiation involve taking limit
* Computer is a finite-precision machine

* Numerical integration
* Stochastic methods with examples from econometrics
  * Simulated method of moments
  * Indirect inference
* Markov Chain Monte Carlo (MCMC)


## Numerical Methods

* Numerical differentiation and integration
* Nothing to do with economics or econometrics

* To find the optimum for the objective function $f:R^K \mapsto R$
by Newton's method, 
  * $K$-dimensional gradient   
  * $K\times K$-dimensional Hessian matrix.

* Programming up the gradient and the Hessian manually is a time-consuming and error-prone job
* Whenever we change the objective function, we have to redo the gradient and Hessian

* More efficient to use numerical differentiation instead of the analytical expressions

The partial derivative of a multivariate function
$f:R^K \mapsto R$ at a point $x_0 \in R^K$ is

$$
\frac{\partial f(x)}{\partial x_k}\bigg|_{x=x_0}=\lim_{\epsilon \to 0}
\frac{f(x_0+\epsilon \cdot e_k) - f(x_0 - \epsilon \cdot e_k)}{2\epsilon},
$$

where $e_k = (0,\ldots,0,1,0,\ldots,0)$ is the identifier of the $k$-th coordinate.

* Numerical execution in a computer follows the basic definition to evaluate
$f(x_0\pm\epsilon \cdot e_k))$ with a small
$\epsilon$. 

* How small is small? Usually we try a sequence of $\epsilon$'s until
the numerical derivative is stable. 

* There are also more sophisticated algorithms.

## Caution

* Numerical methods are not panacea
* Not all functions are differentiable or integrable.
* Before turning to numerical methods, it is always imperative to try to understand the behavior of the function at the first place.
* Some symbolic software, such as `Mathematica` or `Wolfram Alpha`, is a useful tool for this purpose. 
* R is weak in symbolic calculation despite the existence of a few packages for this purpose.

## Stochastic Methods

* An alternative to numerical integration is the stochastic methods.
* The underlying principle of stochastic integration is the law of large numbers.

* Let  $\int h(x) d F(x)$ be an integral where $F(x)$ is a probability distribution.
* Approximate the integral by
$\int h(x) d F(x) \approx S^{-1} \sum_{s=1}^S h(x_s)$, where $x_s$ is randomly
generated from $F(x)$.

* When $S$ is large, a law of large numbers gives

$$
S^{-1} \sum_{s=1}^S h(x_s) \stackrel{\mathrm{p}}{\to} E[h(x)] = \int h(x) d F(x).
$$

* If the integration is carried out not in the entire support of $F(x)$ but on a subset $A$, then

$$
\int_A h(x) d F(x) \approx S^{-1} \sum_{s=1}^S h(x_s) \cdot 1\{x_s \in A\},
$$

where $1\{\cdot\}$ is the indicator function.

* In theory, use an $S$ as large as possible.
* In reality, constrained by the computer's memory and computing time.
* No clear guidance of the size of $S$ in practice. 
* Preliminary experiment can help decide an $S$ that produces stable results.

* Stochastic integration is popular in econometrics and statistics, thanks to its convenience in execution.

### Random Variable Generation

* If the CDF $F(X)$ is known, we can generate random variables that follow such a distribution.
  * Draw $U$ from  $\mathrm{Uniform}(0,1)$
  * Compute $X = F^{-1}(U)$

## Markov Chain Monte Carlo

* If the pdf $f(X)$ is known, we can generate a sample by *importance sampling*
  * [Metropolis-Hastings algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm)
 (MH algorithm) is such a method.
  * MH is one of the [Markov Chain Monte Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo) methods.
* It can be implemented in the R package `mcmc`.
  * [This page](https://chi-feng.github.io/mcmc-demo/) contains demonstrative
examples of MCMC.

### Metropolis-Hastings Algorithm

* Theory of the MH requires long derivation
* Implementation is straightforward.

### Example

Use MH to generate a sample of normally distributed observations with
$\mu = 1$ and $\sigma = 0.5$.

* In the function `metrop`, we provide the logarithm of the density of

$$
\log f(x) = -\frac{1}{2} \log (2\pi) - \log \sigma - \frac{1}{2\sigma^2} (x-\mu)^2
$$

  * The first term can be omitted as it is irrelevant to the parameter.

In [ ]:
h <- function(x, mu = 1, sd = 0.5) {
  y <- -log(sd) - (x - mu)^2 / (2 * sd^2)
  # y <- log( dnorm(x, 1, 0.5) )
} # un-normalized function (doesn't need to integrate as 1)

out <- mcmc::metrop(obj = h, initial = 0, nbatch = 100, nspac = 1)

par(mfrow = c(3, 1))
par(mar = c(2, 2, 1, 1))
plot(out$batch, type = "l") # a time series with flat steps

out <- mcmc::metrop(obj = h, initial = 0, nbatch = 100, nspac = 10)
plot(out$batch, type = "l") # a time series looks like a white noise

out <- mcmc::metrop(obj = h, initial = 0, nbatch = 10000, nspac = 10)
plot(density(out$batch), main = "", lwd = 2)

xbase <- seq(-2, 2, 0.01)
ynorm <- dnorm(xbase, mean = 1, sd = 0.5)
lines(x = xbase, y = ynorm, type = "l", col = "red")


### Outcomes

* The first panel is a time series where
the marginal distribution of each observations follows $N(1,0.5^2)$. 
  * Time dependence is visible
  * flat regions are observed when the Markov chain rejects a new proposal
so the value does not update over two periods. 

* The middle panel collects the time series every 10 observations on the Markov chain
  * serial correlation is weakened
  * No flat region is observed
 
  
* The third panel compares     
  * kernel density of the simulated observations (black curve) 
  * density function of $N(1,0.5^2)$ (red curve).


### Bayesian Inference


* Bayesian framework offers a coherent and natural language for statistical decision. 
* Bayesian views both the data $\mathbf{X}_{n}$ and the
parameter $\theta$ as random variables
* Before observeing the data, researcher holds a *prior distribution* $\pi$ about $\theta$
* After observing the data, researcher updates the prior distribution to a *posterior distribution* $p(\theta|\mathbf{X}_{n})$. 


### Bayes Theorem

* Let $f(\mathbf{X}_{n}|\theta)$ be the likelihood
* Let $\pi$ be the prior

* The celebrated Bayes Theorem is

$$
p(\theta|\mathbf{X}_{n})\propto f(\mathbf{X}_{n}|\theta)\pi(\theta)
$$




### Classical Analytical Example 

* $\mathbf{X}_{n}=(X_{1},\ldots,X_{n})$ is an iid sample drawn from a normal distribution with unknown $\theta$ and known $\sigma$
* If a researcher's prior distribution
$\theta\sim N(\theta_{0},\sigma_{0}^{2})$, her posterior distribution
is, by some routine calculation, also a normal distribution

$$
p(\theta|\mathbf{x}_{n})\sim N\left(\tilde{\theta},\tilde{\sigma}^{2}\right),
$$

where
$\tilde{\theta}=\frac{\sigma^{2}}{n\sigma_{0}^{2}+\sigma^{2}}\theta_{0}+\frac{n\sigma_{0}^{2}}{n\sigma_{0}^{2}+\sigma^{2}}\bar{x}$
and
$\tilde{\sigma}^{2}=\frac{\sigma_{0}^{2}\sigma^{2}}{n\sigma_{0}^{2}+\sigma^{2}}$.


### Bayesian Credible Set

$$
\left(\tilde{\theta}-z_{1-\alpha/2}\cdot\tilde{\sigma},\ \tilde{\theta}+z_{1-\alpha/2}\cdot\tilde{\sigma}\right).
$$

* Posterior distribution depends on $\theta_{0}$ and $\sigma_{0}^{2}$
from the prior. 
* When the sample size is sufficiently, the data overwhelms the prior


### Frequentist Confidence Internval


* $\hat{\theta}=\bar{x}\sim N(\theta,\sigma^{2}/n)$. 

* Confidence interval is

$$
\left(\bar{x}-z_{1-\alpha/2}\cdot\sigma/\sqrt{n},\ \bar{x}-z_{1-\alpha/2}\cdot\sigma/\sqrt{n}\right).
$$

### Comparison

* Bayesian produces a posterior distribution
  * The posterior distribution implies point estimates and credible set
  * Data are fixed (invariant)
  * A prior distribution is needed

* Frequestist produces a point estimator
  * Before data are observed, the point estimator is random
  * Inference is imply by the point estimator **before observation**
  * Only data are realized, the point estimate is a fixed number
  * No prior distribution is needed
  

### Code Example

* Prior distribution $\mu \sim Beta(2,5)$
* [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) is flexible

In [ ]:
# prior distribution
xbase = seq(0.01, 0.99, by = 0.01)
plot( x = xbase, y = dbeta(xbase, 2, 5), type = "l" )


* The nature generates data

In [ ]:
n <- 20
x <- rnorm(n) + 0.9
print(sort(x))

* The researcher correctly specifies the normal model
* Given the data, she infers $\theta$ according to the posterior distribution

In [ ]:
loglik <- function(theta) {   y <- sum( log( dnorm( x, theta, 1 ) ) ) } 

posterior <- function(theta) { loglik(theta) + log(dbeta(theta, 2, 2))  }

nbatch <- 10000
out <- mcmc::metrop(obj = posterior, initial = 0.1, nbatch = nbatch, nspac = 10)$batch
out <- out[-(1:round(nbatch/10))] # remove the burn-in period

plot(density(out))

print( quantile(out, probs = c(0.025, 0.975) ))

### Laplace-type Estimator

* Some criterion functions in econometrics is difficult to optimize
* @chernozhukov2003mcmc's *Laplace-type estimator* replaces optimization by simulation

* Let $L_n(\theta)$ be an criterion function (say, OLS, (negative) log likelihood, etc),

$$
f_n (\theta) = \frac{\exp(-L_n(\theta))\pi(\theta)}{\int_{\Theta} \exp(-L_n(\theta))\pi(\theta)}
$$

and $\pi(\theta)$ be the density of a prior.

* The smaller is the value of the objective function, the larger it weighs. 
* Exponential transformation comes from [Laplace approximation](https://en.wikipedia.org/wiki/Laplace's_method).

* Use MCMC to simulate the distribution of $\theta$.
* For a linear regression model, the code block compares the OLS estimator with the LTE estimator 

In [ ]:
library(magrittr)

# DGP
n <- 100
b0 <- c(.1, .1)
X <- cbind(1, rnorm(n))
Y <- X %*% b0 + rnorm(n)

In [ ]:
# Laplace-type estimator
# L <- function(b) -0.5 * sum((Y - X %*% b)^2) - 0.5 * crossprod(b - c(0, 0))
# notice the "minus" sign of the OLS objective function
# here we use a normal prior around (0,0) with var 1

L <- function(b) -0.5*sum((Y - X %*% b)^2) # flat prior

nbatch <- 10000
out <- mcmc::metrop(obj = L, initial = c(0, 0), nbatch = nbatch, nspac = 20)$batch

# summarize the estimation
bhat2 <- out[-(1:round(nbatch / 10)), 2] # remove the burn in
bhat2_point <- mean(bhat2)
bhat2_sd <- sd(bhat2)
bhat2_CI <- quantile(bhat2, c(.025, .975))

# report results
print(cat(
  "The posterior mean =", bhat2_point, " sd = ", bhat2_sd,
  " \n and C.I.=(", bhat2_CI, ")"
))

plot(density(bhat2), main = "posterior from normal prior")

In [ ]:
# OLS estimation

b2_OLS <- (lm(Y ~ -1 + X) %>% summary() %>% coef())[2, ]
# "-1" in lm( ) because X has contained intercept
print(cat(
  "The OLS point est =", b2_OLS[1], " sd = ", b2_OLS[2],
  " \n and C.I.=(", c(b2_OLS[1] - 1.96 * b2_OLS[2], b2_OLS[1] + 1.96 * b2_OLS[2]), ")"
))

## Reading

CASI: Ch.2.1; 3.3--4; 13.4
